In [14]:
import joblib
import pandas as pd
import sys, pathlib

# Setup ruta al proyecto
PROJECT_ROOT = pathlib.Path().resolve().parent.parent
if str(PROJECT_ROOT) not in sys.path:
    sys.path.insert(0, str(PROJECT_ROOT))

from src import config as cfg

In [16]:
data = joblib.load(cfg.DATA / "processed" / "lstm5d_data.pkl")
df_prices = pd.read_parquet(cfg.DATA / "raw" / "prices.parquet")

X = data["X"]
y = data["y"]

# Filtrar muestras inválidas
import numpy as np
mask = ~np.isnan(y).any(axis=1) & ~np.isinf(y).any(axis=1)
X = X[mask]
y = y[mask]


print("X:", X.shape, "y:", y.shape)


X: (4455, 60, 80) y: (4455, 40)


In [36]:
import numpy as np
import pandas as pd
import joblib
from sklearn.preprocessing import StandardScaler

# Cargar datos
data = joblib.load(cfg.DATA / "processed" / "lstm5d_data.pkl")
X = data["X"]
y = data["y"]


fechas_X = pd.to_datetime(data["dates"])

train_mask = fechas_X < "2019-01-01"
val_mask   = (fechas_X >= "2019-01-01") & (fechas_X < "2021-01-01")
test_mask  = fechas_X >= "2021-01-01"

X_train, y_train = X[train_mask], y[train_mask]
X_val, y_val     = X[val_mask], y[val_mask]
X_test, y_test   = X[test_mask], y[test_mask]

fechas_train = fechas_X[train_mask]
fechas_val   = fechas_X[val_mask]
fechas_test  = fechas_X[test_mask]

print("Train:", X_train.shape, fechas_train.min(), "→", fechas_train.max())
print("Val:  ", X_val.shape,   fechas_val.min(),   "→", fechas_val.max())
print("Test: ", X_test.shape,  fechas_test.min(),  "→", fechas_test.max())


# Paso 1: eliminar muestras con NaNs en X_train antes de escalar
mask_valid = ~np.isnan(X_train).any(axis=(1, 2))
X_train = X_train[mask_valid]
y_train = y_train[mask_valid]

# Paso 2: Escalado de X
X_train_2d = X_train.reshape(-1, X_train.shape[2])
scaler_X = StandardScaler()
X_train_scaled = scaler_X.fit_transform(X_train_2d).reshape(X_train.shape)

X_val_scaled = scaler_X.transform(X_val.reshape(-1, X_val.shape[2])).reshape(X_val.shape)
X_test_scaled = scaler_X.transform(X_test.reshape(-1, X_test.shape[2])).reshape(X_test.shape)

# Paso 3: Escalado de y
scaler_y = StandardScaler()
y_train_scaled = scaler_y.fit_transform(y_train)
y_val_scaled   = scaler_y.transform(y_val)
y_test_scaled  = scaler_y.transform(y_test)

# Guardar escaladores
joblib.dump(scaler_X, cfg.MODELS / "scaler_X_lstm5d.pkl")
joblib.dump(scaler_y, cfg.MODELS / "scaler_y_lstm5d.pkl")



Train: (2091, 60, 80) 2012-08-15 00:00:00 → 2018-12-31 00:00:00
Val:   (731, 60, 80) 2019-01-01 00:00:00 → 2020-12-31 00:00:00
Test:  (1633, 60, 80) 2021-01-01 00:00:00 → 2025-06-21 00:00:00


['C:\\Users\\ferra\\Documents\\TFM\\models\\scaler_y_lstm5d.pkl']

In [38]:
import tensorflow as tf
from tensorflow.keras import layers, models

model = models.Sequential([
    layers.Input(shape=(cfg.WINDOW, X.shape[2])),  # 60 x 80
    layers.LSTM(64, return_sequences=True),
    layers.Dropout(0.2),
    layers.LSTM(32),
    layers.Dense(y.shape[1])  # ← ✅ Solo 40 salidas (1 por activo)
])

model.compile(optimizer=tf.keras.optimizers.Adam(1e-3), loss="mse")
model.summary()


Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm_6 (LSTM)                   │ (None, 60, 64)         │        37,120 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 60, 64)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_7 (LSTM)                   │ (None, 32)             │        12,416 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 40)             │         1,320 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 50,856 (198.66 KB)

 Trainable params: 50,856 (198.66 KB)

 Non-trainable params: 0 (0.00 B)

In [40]:
from tensorflow.keras.callbacks import EarlyStopping

early_stop = EarlyStopping(patience=5, restore_best_weights=True)

print("Check X_train_scaled:", np.isnan(X_train_scaled).sum(), "NaNs /", np.isinf(X_train_scaled).sum(), "Infs")
print("Check y_train_scaled:", np.isnan(y_train_scaled).sum(), "NaNs /", np.isinf(y_train_scaled).sum(), "Infs")
print("Check X_val_scaled:", np.isnan(X_val_scaled).sum(), "NaNs /", np.isinf(X_val_scaled).sum(), "Infs")
print("Check y_val_scaled:", np.isnan(y_val_scaled).sum(), "NaNs /", np.isinf(y_val_scaled).sum(), "Infs")
print("X_train range:", np.min(X_train_scaled), "→", np.max(X_train_scaled))
print("y_train range:", np.min(y_train_scaled), "→", np.max(y_train_scaled))



history = model.fit(
 X_train_scaled, y_train_scaled,
 validation_data=(X_val_scaled, y_val_scaled),
 epochs=50,
 batch_size=32,
 callbacks=[early_stop],
 verbose=1
)



Check X_train_scaled: 0 NaNs / 0 Infs
Check y_train_scaled: 0 NaNs / 0 Infs
Check X_val_scaled: 0 NaNs / 0 Infs
Check y_val_scaled: 0 NaNs / 0 Infs
X_train range: -13.192703 → 44.809296
y_train range: -7.224562 → 20.043701
Epoch 1/50
66/66 ━━━━━━━━━━━━━━━━━━━━ 8s 74ms/step - loss: 1.0183 - val_loss: 2.0568
Epoch 2/50
66/66 ━━━━━━━━━━━━━━━━━━━━ 4s 61ms/step - loss: 0.9052 - val_loss: 2.1346
Epoch 3/50
66/66 ━━━━━━━━━━━━━━━━━━━━ 3s 48ms/step - loss: 0.8255 - val_loss: 2.1164
Epoch 4/50
66/66 ━━━━━━━━━━━━━━━━━━━━ 4s 61ms/step - loss: 0.7874 - val_loss: 2.1125
Epoch 5/50
66/66 ━━━━━━━━━━━━━━━━━━━━ 4s 62ms/step - loss: 0.7235 - val_loss: 2.1357
Epoch 6/50
66/66 ━━━━━━━━━━━━━━━━━━━━ 4s 63ms/step - loss: 0.7192 - val_loss: 2.1572


In [42]:
from pathlib import Path

# Crear carpeta si no existe
Path("models").mkdir(parents=True, exist_ok=True)

model.save(cfg.MODELS / "lstm5d.keras")
print("✅ Modelo guardado en formato .keras")
joblib.dump(history.history, cfg.RESULT / "history_lstm5d.pkl")


✅ Modelo guardado en formato .keras


['C:\\Users\\ferra\\Documents\\TFM\\results\\history_lstm5d.pkl']

In [44]:
y_pred = model.predict(X_test_scaled)
rmse = np.sqrt(((y_test_scaled - y_pred)**2).mean(axis=0))
rmse_mean = rmse.mean()
print("RMSE medio:", rmse_mean)

joblib.dump(rmse_mean, cfg.RESULT / "rmse_lstm5d.pkl")


52/52 ━━━━━━━━━━━━━━━━━━━━ 2s 25ms/step
RMSE medio: 1.203264


['C:\\Users\\ferra\\Documents\\TFM\\results\\rmse_lstm5d.pkl']

El valor de RMSE medio obtenido (1.03) se justifica por la mayor complejidad del modelo LSTM al trabajar con ventanas de 60 días y 80 variables por muestra, lo cual introduce alta dimensionalidad y mayor varianza en los errores. Aunque el rendimiento es inferior al de otras configuraciones con menos variables (RMSE ≈ 0.14), este modelo captura dinámicas más ricas, lo que puede resultar valioso al combinarse con métodos evolutivos en la fase de optimización de carteras.